Notebook para experimentar con el cálculo de la SNR de imágenes EL. Calcular la relación señal-ruido promedio SNR50 según lo definido por IEC NP 60904-13 necesita 2 imágenes EL de referencia y una imagen de fondo

In [1]:
# Importar librerías

import numpy as np
import matplotlib.pyplot as plt
import cv2
from os import listdir, path

In [ ]:
# Función para el cálculo de la relación señal a ruido (SNR) promedio SNR50

def SNR_IEC(i1, i2, ibg=0, allow_color_images=False):
    """
    Calculla la relación señal a ruido (SNR) promedio SNR50 según la norma IEC 60904-13

    Parámetros

    i1: numpy.ndarray
        Imagen de referencia
    i2: numpy.ndarray
        Imagen a comparar
    ibg: float
        Imagen del fondo
    allow_color_images: bool
        Permite el uso de imágenes a color

    Retorna

    snr50: float
        Relación señal a ruido (SNR) promedio SNR50
    """
    # Validar que las imágenes sean del tipo float64 (doble precisión):
    i1 = i1.astype(np.float64)
    i2 = i2.astype(np.float64)
    if ibg != 0:
        ibg = ibg.astype(np.float64)
        assert i1.shape == ibg.shape, "Las imágenes deben tener la misma resolución"
    # Validar que las imágenes tengan la misma resolución:
    if i1.shape != i2.shape:
        raise ValueError("Las imágenes deben tener la misma resolución")
    
    # Validar que las imágenes sean en escala de grises:
    if not allow_color_images:
        if len(i1.shape) == 3:
            raise ValueError("Las imágenes deben ser en escala de grises")
        
    # SNR definido por la norma IEC 60904-13
    signal = 0.5 * (i1 + i2) -ibg
    noise = 0.5**0.5 * np.abs(i1 - i2) * ((2 / np.pi)**0.5)
    signal = signal.sum()
    noise = noise.sum()

    snr50 = signal / noise

    return snr50
    

In [ ]:
# Función para la lectura de imágenes

def read_images(directory, allow_color_images=False):
    """
    Lee todas las imágenes en un directorio

    Parámetros

    directory: str
        Directorio con las imágenes
    allow_color_images: bool
        Permite el uso de imágenes a color

    Retorna

    images: list
        Lista con las imágenes
    """
    # Leer todas las imágenes en el directorio
    images = []
    for f in listdir(directory):
        if f.endswith('.jpg') or f.endswith('.png'):
            image = cv2.imread(path.join(directory, f), cv2.IMREAD_GRAYSCALE)
            if allow_color_images:
                image = cv2.imread(path.join(directory, f), cv2.IMREAD_COLOR)
            images.append(image)
    return images

datasetEL = imágenes originales EL.  
datasetBG = imágenes originales del fondo.

In [ ]:
# Prueba de la función SNR_IEC

# Ruta de las imágenes
image_path = "E:/Panel_30W/P13/8/JPEG_8"
background_path ="E:/Panel_30W/P13/8/fondo_8"

datasetEL = read_images(image_path)
datasetBG = read_images(background_path)

# Imprime el número de imágenes en cada dataset
# para probar que las imágenes se agregaron
print("Imágenes en el dataset EL: ", len(datasetEL))
print("Imágenes en el dataset BG: ", len(datasetBG))